In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
name="farhananis005/lora-qwen-2-7b-Math-e1"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model)

# Math

In [ ]:
math_input=[
"You have five numbers 10, 11, 12, 13, and 14. What is the sum of the largest number and the second smallest number?",

"Find how many multiples of 4 are less than 30.",

"The smallest two-digit number that can be formed by two of 0, 3, 4, 6, and 8 is subtracted by a certain number and it became 7. Find the number.",

"The smallest two-digit number that can be formed by two of 5, 3, 4, 8, and 2 is subtracted by a certain number and it became 7. Find the number.",

"The exchange rate refers to the rate at which one country's currency is exchanged for another country's currency. Raj came back from a trip to the U.K and exchanged the remaining 140 punds for 14000 rupees in Indian money at the bank today. What is the exchange rate of the Indian Rupees to the British pounds today?",

"""The smallest two-digit number that can be formed by two of 3, 4, 0, 6, and 8 is subtracted by a certain number and it became 7. Find the number.
The smallest two-digit number that can be formed by two of the given numbers (0, 3, 4, 6, and 8) is 30, because 0 cannot be the first digit of a two-digit number. If this number (30) is subtracted by a certain number and it became 7, we can set up the following equation to find the certain number: 30 - x = 7 Now, we solve for x: x = 30 - 7 x = 23 So, the certain number is 23.
The smallest two-digit number that can be formed by two of 5, 3, 1, 8, and 9 is subtracted by a certain number and it became 7. Find the number.""",

"""The exchange rate refers to the rate at which one country's currency is exchanged for another country's currency. Soojeong came back from a trip to the U.S. and exchanged the remaining 140 dollars for 158,760 won in Korean money at the bank today. What is the exchange rate of the Korean Won to the US Dollar today?
To find the exchange rate of the Korean Won to the US Dollar, we need to divide the amount of Korean Won Soojeong received by the amount of US Dollars she exchanged. So, the exchange rate (KRW/USD) is: 158,760 KRW / 140 USD = 1,134 KRW/USD Therefore, the exchange rate is 1,134 Korean Won to 1 US Dollar.
The exchange rate refers to the rate at which one country's currency is exchanged for another country's currency. Igor came back from a trip to the U.K and exchanged the remaining 140 pounds for 14000 rubles in at the bank today. What is the exchange rate of the Russian Rubles to the British pounds today?""",

"Danny collects bottle caps. On Monday, he went to the park and found 36 bottle caps but he threw away 45 old ones. On Tuesday, he visited the beach and found another 58 bottle caps, however, he ended up throwing away 30 from his old collection. And then on Wednesday, he went to a concert where he found 80 more bottle caps but he decided to throw away another 70 old ones. After all these, he has 65 bottle caps left. How many more bottle caps did Danny find than those he threw away during these three days?",

"In a school there are 706 girls and 222 boys. 418 more girls joined the school. The school administration requires maintaining a student-teacher ratio of 20:1. After accommodating the new students, how many teachers should be there in the school now?",

"Milan has two phone lines for his long-distance phone service. For the first phone line, he pays a 3 dollar monthly fee plus 15 cents per minute, and for the second phone line, he pays a 4 dollar monthly fee plus 10 cents per minute. Last month, Milan's total long-distance bill was 56.00 dollars for both lines, and he used the first phone line for 20 more minutes than the second phone line. How many minutes in total was Milan billed for on both lines?",

]

In [ ]:
# Path to the output file
import os
output_file = f"{name.split('/')[-1]}math-benchmark.txt"
n=0
full_path = os.path.join("/content", output_file)
# Open the file in write mode
with open(full_path, "w") as f:
    for input in math_input:
        alpaca_prompt = """You are an expert at solving math word problems
### Input:
{}

### Response:
{}"""
        # Prepare the prompt
        prompt = alpaca_prompt.format( input, "")

        # Tokenize input
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        # Generate output
        outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
        output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        n=n+1
        # Write to file
        f.write(f"Question: {n}\n")
        f.write(f"Generated Response: \n {output_text}\n")
        f.write("Correct Answer:\n")
        f.write("Correct Reasoning:\n")
        f.write("Helpfulness:\n\n")

print(f"Results written to {output_file}")

# Bio

In [ ]:
bio_input=[

"Condition: Parkinson's Disease Symptom: Tremor",

"Chemotherapy",

"",

"I have been taking the antidepressant Escitalopram for the last three months, and I am experiencing some issues. I constantly feel tired and drowsy, which affects my daily activities. I have also developed a low sex drive and started to gain weight. Another weird side effect is that I have these sensations of electric shock-like feelings, primarily when I move my head too quickly.",

"Artificial pancreas (AP) systems have shown promise in treating Type 1 diabetes (T1D) by continuously monitoring glucose levels and adjusting insulin delivery. In a 6-month randomized controlled trial, 138 participants with T1D received either closed-loop AP therapy or sensor-augmented pump (SAP) therapy. The primary outcome was the percentage of time in range for glucose levels. Results showed the AP group had a higher percentage of time in range (70.6% vs. 59.4%, P<0.001), lower mean glucose levels (156.8 vs. 173.5 mg/dL, P<0.001), and reduced hypoglycemia events (2.2% vs. 3.6%, P<0.001). There were no significant differences in the incidence of diabetic ketoacidosis or severe hypoglycemia.",

"Medical concept: Antibiotic resistance Patient example: A patient with a recurring urinary tract infection",

"The patient has a history of irritable bowel syndrome and has been experiencing bloating, constipation, and diarrhea.",

"What are the genetic and environmental factors associated with the development of diabetes?",

"Patient is a 5-year-old child with lead exposure levels of 10 mcg/dL.",

"Lab tests: CBC, CMP, lipid panel; Abnormal results: Hemoglobin A1c 8% (indicating uncontrolled diabetes), elevated liver enzymes (ALT 80 U/L, AST 85 U/L), low HDL cholesterol (30 mg/dL)",


]

In [ ]:
bio_instructions=[

"Explain the difference between a medical condition and a related symptom.",

"Describe the benefits and drawbacks of a medical procedure or treatment.",

"Explain why the drug interaction between Warfarin and Vitamin K-containing foods is concerning and how to manage it.",

"From a patient's experience, extract the key symptoms and side effects they are having due to their medication.",

"Summarize the key findings from this abstract of a research article about the treatment of Type 1 diabetes.",

"Given a medical concept or principle, explain how it relates to a real-life situation or patient example.",

"Given a medical case scenario involving a patient with digestive problems, provide recommendations for improving digestion and reducing discomfort.",

"Provide an overview of the current understanding of the genetic and environmental factors associated with the development of diabetes.",

"Given a patient's demographic data and lead exposure levels, assess for potential neurodevelopmental effects and provide appropriate management recommendations.",

"Given a medical report with various lab test results, identify any abnormal values and provide an explanation for their significance.",


]

In [ ]:
# Path to the output file
output_file = f"{name.split('/')[-1]}math-benchmark.txt"
n=0
# Open the file in write mode
with open(full_path, "w") as f:
    for input in math_input:
        alpaca_prompt = """You are an expert at solving math word problems
### Input:
{}

### Response:
{}"""
        # Prepare the prompt
        prompt = alpaca_prompt.format( input, "")

        # Tokenize input
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        # Generate output
        outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
        output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        n=n+1
        # Write to file
        f.write(f"Question: {n}\n")
        f.write(f"Generated Response: \n {output_text}\n")
        f.write("Correct Answer:\n")
        f.write("Correct Reasoning:\n")
        f.write("Helpfulness:\n\n")

print(f"Results written to {output_file}")